# This jupyter notebook is for assistant maintenance purposes only.

In [15]:
# %pip install --upgrade openai

In [16]:
#import the necessary libraries
from openai import OpenAI
from typing_extensions import override
from openai import AssistantEventHandler
from dotenv import load_dotenv
import os

load_dotenv()

True

In [17]:

client = OpenAI()

In [18]:
class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        print(f"\nassistant > ", end="", flush=True)

    @override
    def on_tool_call_created(self, tool_call):
        print(f"\nassistant > {tool_call.type}\n", flush=True)

    @override
    def on_message_done(self, message) -> None:
        # print a citation to the file searched
        message_content = message.content[0].text
        annotations = message_content.annotations
        citations = []
        for index, annotation in enumerate(annotations):
            message_content.value = message_content.value.replace(
                annotation.text, f"[{index}]"
            )
            if file_citation := getattr(annotation, "file_citation", None):
                cited_file = client.files.retrieve(file_citation.file_id)
                citations.append(f"[{index}] {cited_file.filename}")

        print(message_content.value)
        print("\n".join(citations))


# Create the assistant with file_search enabled

Our first step is to create an Assistant that can do file searching regardless of where the vector store resides (Assistant or Thread)

In [19]:
#Creating the Assistant

assistant = client.beta.assistants.create(
    model="gpt-4o",
    
    instructions="""
        You are BisonGPT, a knowledgeable and supportive assistant dedicated to helping students at Howard University. You have access to a wide range of information about Howard, stored in various documents and files, which you can search to provide specific, accurate answers.

        Your main role is to assist students by answering questions, offering academic advice, and helping them navigate university processes. You can:
        - Recommend course schedules tailored to students' academic plans and graduation requirements.
        - Advise on graduation plans and pathways, based on program-specific requirements and deadlines.
        - Explain university policies, including financial aid, academic advising, and admissions.
        - Guide students on campus resources, student organizations, and events.

        When responding, aim to be clear, concise, and supportive. Reference the university's guidelines and the most relevant information available in your files to ensure your answers are both accurate and useful. Adapt your tone to be professional but friendly, as you are here to encourage and empower students on their academic journey.
    """,
    
    name="BisonGPT",
    
    tools=[{"type": "file_search"}],
    
    metadata={
        "can_be_used_for_file_search": "True",
        "can_hold_vector_store": "True",
    },
)

# Create a vector store


In [20]:
vector_store = client.beta.vector_stores.create(name="Howard University Data")

def get_all_files(directory_path):
    file_paths = []
    for root, _, files in os.walk(directory_path):
        for file in files:
            file_paths.append(os.path.join(root, file))
    return file_paths

files = get_all_files("../Data/")

file_streams = [open(path, "rb") for path in files]

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

#close the files

for file in file_streams:
    file.close()

completed
FileCounts(cancelled=0, completed=3, failed=0, in_progress=0, total=3)


# Or update the vector store with new files

Still testing DO NOT RUN THIS

In [ ]:
# #Retrieve and clear out current files

# vector_store_files = client.beta.vector_stores.files.list(vector_store_id=vector_store.id)

# for file in vector_store_files.data:
#     client.beta.vector_stores.files.delete(
#         vector_store_id=vector_store.id, file_id=file.id
#     )

# #Get all the new files

# files = get_all_files("../Data/")

# file_streams = [open(path, "rb") for path in files]

# file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
#     vector_store_id=vector_store.id, files=file_streams
# )

# Attach the vector store to the assisstant

In [21]:
try:
    assistant = client.beta.assistants.update(
        assistant_id=assistant.id,
        tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
    )
except Exception as e:
    print(f"Failed to attach vector store to assistant: {e}")